In [1]:
import os
import math
import dill
from itertools import permutations




import numpy as np
import scipy as sp
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import gridspec
from sklearn.linear_model import LogisticRegression as logreg
from statsmodels.regression.mixed_linear_model import MixedLM


import TwoPUtils
import STX3KO_analyses as stx
u = stx.utilities
ko_mice = stx.ymaze_sess_deets.ko_mice
ctrl_mice = stx.ymaze_sess_deets.ctrl_mice


plt.rcParams['pdf.fonttype']=42
plt.rcParams['font.family'] ='sans-serif'
plt.rcParams['font.serif'] = 'Arial'

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
figdir = "/mnt/BigDisk/YMazeResults/neural/naive_bayes"
os.makedirs(figdir,exist_ok=True)

In [3]:
gamma = lambda x,kappa:  1/sp.special.gamma(kappa)* np.exp(-x)*np.power(x,kappa-1)
poiss = lambda k,lam:  1/sp.special.gamma(k)* np.exp(-lam)*np.power(lam,k)

def logsumexp(x,axis=-1):
    c = x.max(axis = axis, keepdims=True) + 1E-5
    return c + np.log(np.sum(np.exp(x - c ),axis=axis, keepdims=True))

In [5]:
mouse = ctrl_mice[0]
day = 4


sess = u.load_single_day(mouse,day)
fam_trial_mat = 1/sess.scan_info['frame_rate']*sess.trial_matrices['spks'][sess.trial_info['LR']==-1*sess.novel_arm, :, :]
fam_trial_mat = np.nanmean(fam_trial_mat, axis=0)

{'date': '03_12_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 8, 'novel_arm': -1, 'ravel_ind': 4}


In [12]:
x = 1*sess.trial_matrices['bin_edges'][:-1]
x[-1]=sess.trial_matrices['bin_edges'][-1]

act_spline = sp.interpolate.interp1d(x, fam_trial_mat.T, axis=1)

In [13]:
start, stop = sess.trial_start_inds[0], sess.teleport_inds[0]

t = sess.timeseries['t'][0,start:stop]
y = 1/sess.scan_info['frame_rate']*sess.timeseries['spks'][:,start:stop]
print(y.shape, t.shape)

y_lambda = act_spline(t)
print(y_lambda.shape)

xtest = 1*sess.timeseries['bin_centers']
likelihood = poiss

(962, 233) (233,)
(962, 233)


In [4]:
# within day models

def crossval(sess, n_cells = -1, poisson = True, nov = False):
    
    # assuming uniform prior over bins
    if nov:
        trial_mask = sess.trial_info['LR']==sess.novel_arm
    else:
        trial_mask = sess.trial_info['LR']==-1*sess.novel_arm
        
    trial_mat = sess.trial_matrices['spks'][trial_mask,:,:]*10
    trial_mat -= np.amin(trial_mat,axis=-1,keepdims=True)
    trial_mat += 1E-3

    # select cells
    if n_cells ==-1:
#         print('all_cells')
        pass
    else:
        rng = np.random.default_rng()
        trial_mat = trial_mat[:,:,rng.permutation(trial_mat.shape[-1])[:n_cells]]
    
    posterior = np.zeros([trial_mat.shape[0], trial_mat.shape[1], trial_mat.shape[1]])
    for trial in range(trial_mat.shape[0]):
        
        mask = np.zeros((trial_mat.shape[0],))<1
        mask[trial] = False
        
        trial_mat_mean = np.nanmean(trial_mat[mask,:,:],axis=0)
        
        y = np.copy(trial_mat[trial,:,:])
        y[np.isnan(y)]=1E-3
        log_likelihood = 0
        for cell in range(trial_mat.shape[-1]):
            if poiss:
                log_likelihood += np.log(poiss(y[:,cell:cell+1], trial_mat_mean[:,cell:cell+1].T)+1E-3)
            else:
                log_likelihood += np.log(gamma(y[:,cell:cell+1], trial_mat_mean[:,cell:cell+1].T)+1E-3)
        
        posterior[trial,:,:] = np.exp(log_likelihood - logsumexp(log_likelihood))
        

    
    return posterior


def test(_y, trial_mat_mean, poisson=False):
    log_likelihood = 0
    for cell in range(trial_mat_mean.shape[-1]):
        yc, tmm = _y[:,cell], trial_mat_mean[:,cell]
        if poisson:
            log_likelihood += np.log(poiss(yc[:,np.newaxis], tmm[np.newaxis,:])+1E-3)
        else:
            log_likelihood += np.log(gamma(yc[:,np.newaxis], tmm[np.newaxis,:])+1E-3)
    return np.exp(log_likelihood - logsumexp(log_likelihood))